In [1]:
import pandas as pd
import tqdm
import os

In [2]:
os.environ['HOME'] = os.getenv('HOME') + '/saturi_lab_multi_nmt_low_resource'
path = os.getenv('HOME')
# df = pd.read_csv(path +'/data/raw/dataset_full_v1.csv')
df = pd.read_csv(path +'/data/raw/dataset_validation.csv')

In [3]:
#weighted sampling for valid set
rw = len(df)/df.reg.value_counts()
df['rw'] = df.reg.map(rw)
df = df.sample(50000, weights='rw')


In [5]:
import re

In [6]:
#data cleaning

df['text']=df['text'].apply(lambda x : re.sub('[\(\)]','',str(x))) # remove parenthesis
df['text']=df['text'].apply(lambda x : re.sub('(\&+[a-zA-Zㄱ-힣\&]+)','',str(x))) # remove words like &&name, &company name, etc.
df['text']=df['text'].apply(lambda x : re.sub('(\@+[ㄱ-힣\@]+)','',str(x))) # remove words like &&name, &company name, etc.
df['text']=df['text'].apply(lambda x : re.sub('(x+[a-zㄱ-힣]+)','',str(x))) # remove words like xx하면, xxxx, etc
df['text']=df['text'].apply(lambda x : re.sub('(\-+[a-zㄱ-힣\-]+)','',str(x))) # remove words like --하면, -시사- etc
df['text']=df['text'].apply(lambda x : re.sub(r'(.+)\1{2,}',r'\1',str(x))) # remove repeating words
df['text']=df['text'].apply(lambda x : re.sub('([ㄱ-힣]+)\~$',r'\1.',str(x))) # sub words ending in ~ to . ex 그럴거야~ 그럴거야.
df['text']=df['text'].apply(lambda x : re.sub('([ㄱ-힣]+)\~','',str(x))) # remove expressions like 어~ 아~ 으~ 오~

df['dial']=df['dial'].apply(lambda x : re.sub('[\(\)]','',str(x)))
df['dial']=df['dial'].apply(lambda x : re.sub('(\&+[a-zA-Zㄱ-힣\&]+)','',str(x)))
df['dial']=df['dial'].apply(lambda x : re.sub('(\@+[ㄱ-힣\@]+)','',str(x))) # remove words like @이름,@주소,@상호명, etc.
df['dial']=df['dial'].apply(lambda x : re.sub('(x+[a-zㄱ-힣]+)','',str(x))) # remove words like xx하면, xxxx, etc
df['dial']=df['dial'].apply(lambda x : re.sub('(\-+[a-zㄱ-힣\-\&]+)','',str(x))) # remove words like --하면, -시사- etc
df['dial']=df['dial'].apply(lambda x : re.sub(r'(.+)\1{2,}',r'\1',str(x))) # remove repeating words
df['dial']=df['dial'].apply(lambda x : re.sub('([ㄱ-힣]+)\~$',r'\1.',str(x))) # sub words ending in ~ to . ex 그럴거야~ 그럴거야.
df['dial']=df['dial'].apply(lambda x : re.sub('([ㄱ-힣]+)\~','',str(x))) # remove expressions like 어~ 아~ 으~ 오~

#remove from training

df.loc[df['text'].str.contains('/'),'rm'] = 1 #containing mislabelled data
df['len'] = df['text'].apply(lambda x: len(x))
df.loc[df['len'] < 2,'rm'] = 1 #length is less than 2
df.drop_duplicates(subset='dial', inplace=True) # drop duplicated senteces in dial
clean_df = df.loc[df['rm']!=1,['text','dial','reg','pair']]

In [7]:
clean_df.head()

,text,dial,reg,pair
310522,그날 근데 조금,그날 근데 쫌,cc,(쫌)/(조금)
110493,DD09근데 이제 빨리 은퇴를 하지.,DD09근데 인자 빨리 은퇴를 하지.,jd,(인자)/(이제)
292695,근데 너무 적극적이어 가지고 어.,근데 너무 적극적이어 가주구 어.,cc,(가주구)/(가지고)
113088,SD206예 그래요. 좀 좀 가볍게 보여요.,SD206예 그러요. 좀 좀 가볍게 보여요.,jd,(그러요.)/(그래요.)
5257,거기 있는 거기 나오는 어쨌든 그 음식들 그런 음식들을 다 한 개씩 가지고 와요.,거기 있는 거기 나오는 어쨌든 그 음식들 그런 음석들을 다 한 개씩 가지고 와요.,kw,(음석들을)/(음식들을)


In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
ckpt = 'circulus/kobart-trans-ko-en-v2'
tokenizer = AutoTokenizer.from_pretrained(ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(ckpt)
translator = pipeline("translation", model=ckpt, max_length=60)

/home/saturilab/anaconda3/envs/saturi_torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [9]:
text = clean_df.text.values.tolist()
len(text)

49617

In [10]:
from torch.utils.data import DataLoader

dataloader = DataLoader(text,  batch_size=512)

In [11]:
data_processed_path = path  + '/data/processed/'
translation_file = data_processed_path + input() # translation file name
outputs = []
for n,i in enumerate(tqdm.tqdm(dataloader)) :
    inputs = tokenizer(i, return_tensors="pt", padding=True).input_ids
    out = model.generate(inputs,num_beams=3, max_new_tokens=60) # do_sample=True, top_k=30, top_p=0.95,
    for j in out :
        result = tokenizer.decode(j, skip_special_tokens=True)
        outputs.append(result)
    
    if n % 100 == 0 :
        with open(translation_file,'w') as f :
            for sen in outputs :
                f.write(sen+'\n')
                
with open(translation_file,'w') as f :
    for sen in outputs :
        f.write(sen+'\n')

100%|██████████| 97/97 [1:07:47<00:00, 41.93s/it]


In [12]:
with open(translation_file,'r') as f :
    raw = f.read().splitlines()

In [13]:
translated_train_data = data_processed_path+input() #file name for translated trained data
clean_df['eng'] = raw
clean_df.to_csv(f'{translated_train_data}.csv', index=False)

In [14]:
tx = pd.read_csv(f'{translated_train_data}.csv')
tx.head()

,Unnamed: 0,text,dial,reg,pair,eng
0,310522,그날 근데 조금,그날 근데 쫌,cc,(쫌)/(조금),But that day is a little bit
1,110493,DD09근데 이제 빨리 은퇴를 하지.,DD09근데 인자 빨리 은퇴를 하지.,jd,(인자)/(이제),Its DD09 but now you retire quickly
2,292695,근데 너무 적극적이어 가지고 어.,근데 너무 적극적이어 가주구 어.,cc,(가주구)/(가지고),But he was so active so huh
3,113088,SD206예 그래요. 좀 좀 가볍게 보여요.,SD206예 그러요. 좀 좀 가볍게 보여요.,jd,(그러요.)/(그래요.),Its like SD206 it looks a little light
4,5257,거기 있는 거기 나오는 어쨌든 그 음식들 그런 음식들을 다 한 개씩 가지고 와요.,거기 있는 거기 나오는 어쨌든 그 음식들 그런 음석들을 다 한 개씩 가지고 와요.,kw,(음석들을)/(음식들을),There Anyway bring all those foods one by one
